# Decision Tree Implementation
Tree-based methods partition the feature space into a set of rectangles, and then fit a simple model (like a constant) in each one. 
To simplify the process, imagine it as recursive binary partitions. We first splits the space first into two regions, the model the response by the mean/majority of Y in each region. Then one of both of these regions are split into two regions, and this process continues until some stopping rule. The preferred stopping rule is when some minimum node size (say 5) is reached.

When when we grow the tree, we need to decide the splitting variables and split points based on impurity of the data points on each node. 
For regression we used the squared-error node impurity measure. For classification, there are different measures:
- Misclassification error
- Gini index
- cross-entropy or deviance
while the last two are differential and preferred to be used in splitting variables.

Not implemented: Pruning.
Cost complexity pruning. At each pruning iteration, a subtree is chosen to be removed and replaced by a leaf node. The choice is based not to hurt prediction accuracy. 

### Time complexity
m is number of features, n is number of observations, d is depth = O(logn) for balanced tree

- Training (batch): most efficient is O(mnd). I think my implementation is O(mn^2d)
- Inferencing: O(d)

In [1]:
import math
import numpy as np
from collections import Counter

In [2]:
class DecisionNode():
    """
    Class that represents a decision node or leaf in the decision tree
    """
    def __init__(self, feature_i=None, threshold=None, value=None, left=None, right=None):
        self.split_feature = feature_i  
        self.split_threshold = threshold
        self.value = value  # Value if the node is a leaf in the tree
        self.left = left    # 'Left' subtree. feature_i <= threshold
        self.right = right  # 'Right' subtree

In [3]:
class DecisionTree():
    def __init__(self, tree_type, min_samples_split=5, max_depth=float("inf"), min_impurity=1e-7):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.tree_type = tree_type
        self.min_impurity = min_impurity
        self.root = None
        
    def _calculate_entropy(self, labels):
        total_count = (len(labels))
        class_probabilities = [cnt / total_count for cnt in Counter(labels).values()]
        return sum(-p * math.log(p, 2) for p in class_probabilities if p>0)
    
    def _calculate_impurity(self, R1, R2):
        # calculate impurity of two branch, rather than information gain and variance reduction
        # which is parent measure minus impurity value calculated below
        y1 = R1[:,-1]
        y2 = R2[:,-1]
        
        total_cnt = len(y1) + len(y2)
        p1 = len(y1) / total_cnt
        p2 = 1- p1
        
        if self.tree_type == 'regression':
            weighted_variance = p1 * np.var(y1) + p2 * np.var(y2)
            return weighted_variance
        
        if self.tree_type == 'classification':
            partition_entropy = p1* self._calculate_entropy(y1) + p2 *self._calculate_entropy(y2)
            return partition_entropy
    
        
    def _calculate_leaf_value(self, y):
        if self.tree_type == 'regression':
            # average
            return np.mean(y)
        if self.tree_type == 'classification':
            # majority vote
            return Counter(y).most_common(1)[0][0]
        
    def _grow_tree(self, X, y, depth):
        
        lowest_impurity = float('Inf')
        best_split =  None
        best_regions = None
        num_features = X.shape[1]
        
        # find best split
        for i in range(num_features):
            feature = X[:,i]
            unique_values = np.unique(feature)
            Xy = np.concatenate((X, y.reshape((len(y), 1))), axis=1)
            
            for thresh in unique_values:
                R1 = Xy[feature <= thresh]
                R2 = Xy[feature > thresh]
                impurity = self._calculate_impurity(R1, R2)
                
                if impurity < lowest_impurity:
                    lowest_impurity = impurity
                    best_split = {'feature_i': i, "threshold": thresh}
                    best_regions = {'R1': R1, 'R2': R2}
                
        treeNode = DecisionNode(feature_i=best_split['feature_i'], 
                                threshold=best_split['threshold'])
        
        R1, R2 = best_regions['R1'], best_regions['R2']
        print(f'Best split is at feature {i}, value {thresh} and the impurity score of this split is {impurity}')
        print(f'The best split splits data to two partitions of size {R1.shape[0]} and {R2.shape[0]}')
        # a leaf
        if depth == self.max_depth or lowest_impurity< self.min_impurity or \
            R1.shape[0] < self.min_samples_split or R2.shape[0] < self.min_samples_split:
            treeNode.value = self._calculate_leaf_value(y)
            return treeNode
           
        # a decision node
        if R1.shape[0] >= self.min_samples_split:
            treeNode.left = self._grow_tree(R1[:,:-1],R1[:,-1], depth+1)
        
        if  R2.shape[0] >= self.min_samples_split:
            treeNode.right = self._grow_tree(R2[:,:-1],R2[:,-1], depth+1)
            
        return treeNode
        
    def fit(self, X, y):
        self.root = self._grow_tree(X, y,0)
        
        
    def predict_value(self, x, node=None):
        if not node:
            node = self.root
        
        if node.value is not None:
            return node.value
        
        if x[node.split_feature] <= node.split_threshold:
            return self.predict_value(x, node.left)
        else:
            return self.predict_value(x, node.right)
        
    
    def predict(self, test_X):
        y_pred = [self.predict_value(x) for x in test_X]
        return y_pred

# Test on sklearn iris dataset

In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [5]:
model = DecisionTree("classification", min_samples_split=3, max_depth=3)
model.fit(X_train, y_train)

Best split is at feature 3, value 2.5 and the impurity score of this split is 1.5834545859901241
The best split splits data to two partitions of size 35 and 77
Best split is at feature 3, value 0.6 and the impurity score of this split is 0.0
The best split splits data to two partitions of size 1 and 34
Best split is at feature 3, value 2.5 and the impurity score of this split is 0.9998783322990061
The best split splits data to two partitions of size 35 and 42
Best split is at feature 3, value 1.7 and the impurity score of this split is 0.18717625687320816
The best split splits data to two partitions of size 34 and 1
Best split is at feature 3, value 2.5 and the impurity score of this split is 0.5266170655714282
The best split splits data to two partitions of size 16 and 26
Best split is at feature 3, value 2.4 and the impurity score of this split is 0.8960382325345575
The best split splits data to two partitions of size 6 and 10
Best split is at feature 3, value 2.5 and the impurity sc

In [7]:
acc = sum(y_test == model.predict(X_test)) / len(y_test)
print(f'accuracy on the test set is {acc}')

accuracy on the test set is 0.9736842105263158


# Tree Bagging
A complicated decision tree will have a low bias but high variance. Bagging is effective in reducing variance as it averages estimates of models fitted on a large of bootstrap samples. 

Out-of-Bag Error Estimation. On averate, each bagged tree makes use of around two-thirds of the observations (The chance of one point not selected in any n draw from n samples with replacement is (1-1/n)^n = 1/e is about 1/3 when n goes to infinity). The remaining one-third of the observations not used is OOB. For each OOB sample, average (regression) all predictions. The resulting OOB error is a valid estimate of the test error for the bagged model.

Feature importance is measured by mean drop is RSS or Gini index / entropy over all trees.

# Random Forest
Random Forest improves over bagged trees by way of a small tweak that decorrelates the trees. That's, at each split, only a random sample m=sqrt(p) predictors is chosen for consideration, so each tree will not be similar/correlated.

# Boosting
Trees are built sequentially. In each step, we fit a tree using the current residuals, rather than the outcome Y, and add the new decision tree into the fitted function in order to update the residuals. We slowly improve f in areas where it does not perform well. Smaller trees are typically sufficient.